# This is one of my backups - I'm struggling to find the final version I had, could be that all copies were in places associated with my Cranfield credentials.

*The functions are all present and correct, but its not my tidiest work - I could probably spend some more time on sorting that out when I take some holiday, an I'm happy to answer questions*

In [74]:
import pandas as pd
import os
import glob
import numpy as np
import scipy.linalg
from scipy.stats import t as tdstr
from scipy.stats import norm
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib
matplotlib.rc('font', size=14)
import matplotlib.pyplot as plt
import numpy.matlib as npm 
%matplotlib inline
from numpy.fft import *

# Denoising MODIS data using FFT 
![Denoising Algorithm](img/picture.png)

This is something I came up with for this project to retain enough data points to use ***It has not been that thouroughly tested or "approved" as a method for cleaning an NDVI time series***

In [84]:
# denoise using FFT, if not using daily data, threshold should be multiplied by the sampling interval 

def ff_denoise(signal, threshold=2):
    fourier = rfft(signal)
    
    frequencies = rfftfreq(signal.size,d=1/365.25)
    
    
    fourier[frequencies > threshold] = 0
    
    return irfft(fourier,len(signal))




###------THE PLAN-----####

# 1.take an NDVI time series (NDVI) and its sampling frequency (gap)
# 2.interpolate across missing values (otherwise the fft just gives us NaN+NaN i for everything)
# 3.apply fft_denoise() (frequency = 2 cycles per year (frequency*gap))
# 4.drop everything more than "pad" below the curve

# 5. repeat steps 2-4 for a set number of iterations, or until no values are dropped, whichever comes first
# Note: currently we're dropping and reinterpolating all previously dropped values

###---INPUTS-----####

# NDVI : NDVI time series
# GAP : sampling frequency of the NDVI time series
# FREQUENCY : threshold frequency that is used to determine high frequency variation due to noise/clouds
# PAD : Threshold below the smoothed waveform - values < smooth wave - pad are dropped at each iteration
# iterations : Max number of iterations - will stop early if no values are dropped in consecutive iterations



def ff_iterdenoise(NDVI, gap=1, frequency=2, pad=0.1,iterations=10):
    
    # want to take a series or array, return a DF with filtered results +keep originals
    s_NDVI=pd.DataFrame({"raw":NDVI,"filtered":NDVI})
    
    # c is where we're going to keep track of the filtered NDVI values, as well as on which iteration they were dropped
    s_NDVI["c"]=-1
    s_NDVI.loc[s_NDVI.raw.isna(),"c"]=0 # already missing values
    
    # this is a bit dodgey, but what we're doing is appending the count of '-1's in c, and looping until it doesn't change
    li=[-1,-10]  
    i=0
    while li[i] != li[i+1] and i<iterations:
        
        # interpolating
        Y = s_NDVI['filtered'].values
        x =s_NDVI.index.copy()

        indices = ~np.isnan(Y)

        yinterp = np.interp(x[~indices], x[indices], Y[indices])
        Y[~indices] = yinterp
        
        # applying the ff filter from earlier, calculating "error"
        
        ff=ff_denoise(s_NDVI.filtered,frequency*gap)

        s_NDVI["ff"]=ff

        s_NDVI["delta"]=s_NDVI.ff-s_NDVI.filtered


        s_NDVI.loc[s_NDVI.delta>pad,"c"]=i+1  # these are the values below our threshold

##---can uncomment this and plot each time if we want to ----##         

#         fig = plt.figure(figsize=(16, 6))
#         ax1 = plt.subplot(111)
#         ax1.plot(s_NDVI.index,s_NDVI.ff,"-", c='k')
#         s=ax1.scatter(s_NDVI.index,s_NDVI.filtered, c=s_NDVI.c, cmap="Set1",s=10)
# #        ax1.plot(sixteen_d, ls="--", marker="x",c="b")
#         fig.colorbar(s)
#         ax1.set_ylabel('NDVI')
#         ax1.set_ylim([-0.1,1])
#         ax1.set_xlim([datetime(2016,1,1),datetime(2020,1,1)])  # <- to view a smaller time period


        s_NDVI.loc[s_NDVI.c!=-1,"filtered"]=np.nan 

        li.append(s_NDVI.c.value_counts()[-1])
        i+=1
        
    s_NDVI.drop("delta",axis=1, inplace=True)    #don't need this
    return s_NDVI

In [81]:
## We need a run of consecutive NDVI values at the start or the model fails ##
# this lets us identify the longest run in the first "days" days so we can use it as a start point

def get_con(NDVI,days=365):
    a = NDVI[:days]  
    m = np.concatenate(( [True], np.isnan(a), [True] ))  # Mask
    ss = np.flatnonzero(m[1:] != m[:-1]).reshape(-1,2)   # Start-stop limits
    start,stop = ss[(ss[:,1] - ss[:,0]).argmax()]
    return a.index[start],stop-start

# These are the functions from the original paper

- I've added the "gap" paramater where necessary so that we can work with daily MODIS data vs just the 16 day Landsat they used

- I've also changed the way "delta" (the discount factor works) we're doing component discounting rather than using a single discount factor like the California paper did (some discussion about why in my write up)

- I've fixed "delvar" Dan - we briefly discussed this over zoom at one point, I'm convinced the way this was originally implemented is incorrect - discounting the degrees of freedom the way it is implemented in the original code does not do what they say it does

- The commented out lines of code were used to plot mortality and droughts etc in the original paper. I didn't use them, but I've kept them there because they were in the original

- The rest of the comments are from the original author(s)

In [1]:
class Prior:
    def __init__(self, m, C, S, nu): 
        self.m = m # mean of t-distribution 
        self.C = C # scale matrix of t-distribution
        self.S = S # precision ~ IG(nu/2,S*nu/2)
        self.nu = nu # degree of freedom

class Model:
    def __init__(self,Y,X,rseas,delta):
        self.Y = Y
        self.X = X
        self.rseas = rseas
        #dd = np.ones(4)*delta   # <- MARKO CHANGED THIS BIT
        self.delta = delta
        ntrend = 2;nregn = X.shape[1]; pseas = len(rseas);nseas = pseas*2;
        m = np.zeros([ntrend+nregn+nseas,1])
        C = scipy.linalg.block_diag(1*np.eye(ntrend),1*np.eye(nregn),1*np.eye(nseas))
        S = np.power(0.2,2); nu = ntrend+nregn+pseas;
        pr = Prior(m,C,S,nu)
        self.prior = pr
        
  


def forwardFilteringM(Model,gap):
# All the parameters estimated here correspond Eqs. 13-16 and the related ones in the Supplementary Information of Liu et al. (2019)
# notation in the code -> notation in Liu et al., 2019: 
# m -> m_t; C -> C_t^{**}; nu -> n_t; 
# a -> a_t; R -> R_t^{**}; F -> F_t; e -> e_t; y -> y_t; Q -> q_t^{**}; f -> f_t; S -> s_t = d_t/n_t

  Y = Model.Y
  X = Model.X
  rseas = Model.rseas
  delta = Model.delta
  Prior = Model.prior
  period = 365.25/gap
  deltrend = delta[0];delregn = delta[1];delseas = delta[2];delvar = 0.999
  
  Ftrend = np.array([[1],[0]]);ntrend = len(Ftrend); Gtrend = np.array([[1,1],[0,1]]);itrend = np.arange(0,ntrend)
  nregn = X.shape[1];Fregn = np.zeros([nregn,1]);Gregn=np.eye(nregn);iregn = np.arange(ntrend,ntrend+nregn)
  pseas = len(rseas);nseas = pseas*2;iseas = np.arange(ntrend+nregn,ntrend+nregn+nseas)
  Fseas = npm.repmat([[1],[0]],pseas,1);Gseas = np.zeros([nseas,nseas]);
  for j in range(pseas):
      c = np.cos(2*np.pi*rseas[j]/period);
      s = np.sin(2*np.pi*rseas[j]/period);
      i = np.arange(2*j,2*(j+1))
      Gseas[np.reshape(i,[2,1]),i] = [[c,s],[-s,c]]
  F = np.concatenate((Ftrend,Fregn,Fseas),axis=0)
  G = scipy.linalg.block_diag(Gtrend,Gregn,Gseas) 
  m = Prior.m; C = Prior.C; S = Prior.S; nu = Prior.nu

  T = len(Y)
  sm = np.zeros(m.shape)
  sC = np.zeros([C.shape[0],C.shape[1],1])
  sS = np.zeros(1)
  snu = np.zeros(1)
  slik = np.zeros(1)
  
  for t in range(T):
      a = np.dot(G,m)
      R = np.dot(np.dot(G,C),np.transpose(G))
      R[np.reshape(itrend,[-1,1]),itrend] = R[np.reshape(itrend,[-1,1]),itrend]/deltrend
      R[np.reshape(iregn,[-1,1]),iregn] = R[np.reshape(iregn,[-1,1]),iregn]/delregn
      R[np.reshape(iseas,[-1,1]),iseas] = R[np.reshape(iseas,[-1,1]),iseas]/delseas
      nu = delvar*nu
      F[iregn,0] = X[t,]

      A = np.dot(R,F);Q = np.squeeze(np.dot(np.transpose(F),A)+S); A = A/Q; f = np.squeeze(np.dot(np.transpose(F),a))
      y = Y[t]

      if ~np.isnan(y):
          e = y-f; ac = (nu+np.power(e,2)/Q)/(nu+1)
          rQ = np.sqrt(Q)
          mlik = tdstr.pdf(e/rQ,nu)/rQ
          m = a+A*e; C = ac*(R-np.dot(A,np.transpose(A))*Q); nu = nu+1; S = ac*S; 
          # About "S = ac*S" (using the notations in Liu et al. (2019)): 
          # s_t = d_t/n_t = (d_{t-1}+e_t^2/(q_t^{**}/s_t))/n_t = s_{t-1} * (n_{t-1}+e_t^2/(q_t^{**})/n_t = ac * s_{t-1}
      else:
          m = a; C = R;
          if t<T-1:
              X[t+1,0] = f
          mlik = np.nan
      sm = np.concatenate((sm,m),axis=1)
      sC = np.concatenate((sC,np.reshape(C,[C.shape[0],C.shape[1],1])),axis=2)
      snu = np.concatenate((snu,[nu]),axis=0)
      sS = np.concatenate((sS,[S]),axis=0)
      slik = np.concatenate((slik,[mlik]),axis=0)
      
  return {'sm':sm, 'sC':sC ,'snu':snu,'slik':slik} 

def computeAnormaly(CLM,AvgCLM,date0,gap):
  deltaT = timedelta(days=gap)
  anCLM = np.zeros([1,CLM.shape[1]])
  for i in range(CLM.shape[0]):
    st = date0+deltaT*(i)
    st = st.timetuple().tm_yday
    et = date0+deltaT*(i+1); et = et.timetuple().tm_yday               
    if et<st:
        window = np.concatenate((np.arange(st,365),np.arange(0,et)))
    else:
        window = np.arange(st,et)
    window[window==365] = 0  # leap year
    anCLM = np.concatenate((anCLM,np.reshape(CLM[i,:]- np.mean(AvgCLM[window,:],axis = 0),[1,CLM.shape[1]])),axis=0)
    
  return anCLM[1:,:]

def Index_low(nn,date0,percentile,gap):
  intervel = gap
  date0_num = date0.toordinal()
  dd = np.arange(date0,date0+timedelta(days=intervel)*len(nn),timedelta(days=intervel))
  dd_num = np.arange(date0_num,date0_num+intervel*(len(nn)),intervel)
  idq = [i for i in range(len(nn)) if np.isfinite(nn[i])] 
  tt1_num = np.arange(dd_num[idq[0]],dd_num[idq[-1]],1)
  f_itp = interp1d(dd_num[idq], nn[idq],kind = 'linear')
  nn_itp = f_itp(tt1_num)

  yday = np.array([date.fromordinal(tt1_num[i]).timetuple().tm_yday for i in range(len(tt1_num))])

  ndvi_mean = np.array([np.mean(nn_itp[yday==i]) for i in range(1,366)])
  ndvi_std = np.array([np.std(nn_itp[yday==i]) for i in range(1,366)])
  if len(ndvi_mean)==365:
    ndvi_mean = np.concatenate((ndvi_mean,[ndvi_mean[-1]]),axis = 0)
    ndvi_std = np.concatenate((ndvi_std,[ndvi_std[-1]]),axis = 0)

  tt2 = np.arange(dd[0],dd[-1],timedelta(days=1))
  tt2_num = np.arange(dd_num[0],dd_num[-1],1)
  yday2 = np.array([date.fromordinal(tt2_num[i]).timetuple().tm_yday for i in range(len(tt2_num))])
  nv = norm.ppf(1-(1-percentile)/2)
  lowboundary = np.array([ndvi_mean[yday2[i]-1]-nv*ndvi_std[yday2[i]-1] for i in range(len(tt2))])

  index_low = [i for i in range(len(dd)) if (~np.isnan(nn[i])) and (nn[i]<lowboundary[tt2_num==dd_num[i]])] 
  return index_low

def PlotEWS(N,date0,sm,sC,snu,gap):
  # thresholds for identification of abnormally high autocorrelation (EWS)  
  # and abnormally low NDVI(ALN)
  quantile1 = 0.95
  quantile2 = 0.80 

  steps = [date0+relativedelta(days=gap*i) for i in range(len(N))]
  lown = Index_low(N,date0,quantile2,gap)
  lown_continuous = []
  for i in range(len(lown)):
    tile = [j for j in lown if (j<=lown[i] and j>=lown[i]-int(90/gap))] 
    if len(tile)>2: 
        #NDVI being abnormally low fro more than half of the time within 3 mon
        lown_continuous = np.concatenate([lown_continuous,[lown[i]]])
        lown_continuous = np.array(lown_continuous).astype(int)
  tmp = np.array([steps[i] for i in lown_continuous])
  #diebackdate = tmp[0]

  steps = np.array(steps)

  xpos = date0
  xtick = [date0+relativedelta(years=2*i) for i in range(0,9)]


  plt.figure(figsize=(16, 12))
  ax1 = plt.subplot(211)

  xlim = [datetime(date0.year,1,1),datetime(2020,1,1)]
  ylim = [0,1]
  ax1.plot(steps,N,'o-k',markersize=5,label='NDVI')
  ax1.plot(steps[lown_continuous],N[lown_continuous],'or',label='ALN')

  # ax1.axvspan(datetime(2007,1,1), datetime(2010,1,1), color='brown', alpha=0.1, lw=0)
  # ax1.axvspan(datetime(2011,1,1), datetime(2016,1,1), color='brown', alpha=0.1, lw=0)

  #xtick = [date0+relativedelta(years=2*i) for i in range(0,2)]
  #ax1.set_xticks(xtick,('00','02','04','06','08','10','12','14','16'))

  ax1.set_ylim(ylim)
  ax1.set_xlim(xlim)
  ax1.set_ylabel('NDVI')
  ax1.legend(loc = 'lower left',ncol=2)
  # ax1.text(xpos, 0.82, '(a)',fontsize=20)
  #ax1.set_xticks(xtick)
  plt.setp(ax1.get_xticklabels(), visible=False)

  warmup = int(730/gap)
  bd = list(map(lambda m,C,nu: m+C*tdstr.ppf(quantile1,nu),sm,sC,snu))
  bd2 = list(map(lambda m,C,nu: m+C*tdstr.ppf(quantile2,nu),sm,sC,snu))

  mbd = np.median(bd[warmup:])
  ews = np.array([i for i,im in enumerate(sm) if im >mbd])
  ews = ews[ews>warmup]
  ews_continuous = []
  window = int(90/gap) # three months

  for i in range(len(ews)):
    tile = [j for j in ews if (j<=ews[i] and j>=ews[i]-window)]
    if len(tile)>window-1:
        ews_continuous = np.concatenate([ews_continuous,[ews[i]]])
  ews_continuous = np.array(ews_continuous).astype(int)
  tmp = steps[ews_continuous]
  #ewsdate = tmp[tmp>datetime(2012,7,15)][0]
  #mortdate = datetime(2015,7,15)
  arrowprops=dict(facecolor='black', shrink=0.05,width=1,headwidth=10)
  ax2 = plt.subplot(212)
  ylim = [-1,1]
  ax2.plot(steps[1:], sm[1:], lw=2, label='mean')
  ax2.fill_between(steps, 2*sm-bd, bd, facecolor='0.7',label=str(int(quantile1*100))+'% range')
  ax2.fill_between(steps, 2*sm-bd2, bd2, facecolor='0.5',label=str(int(quantile2*100))+'% range')
  ax2.plot([steps[warmup],steps[-1]],[mbd,mbd],'--',color='0.4')
  ax2.plot(steps[ews_continuous],sm[ews_continuous],'^r',markersize=3,label='EWS')
  #  ax2.axvspan(datetime(2007,1,1), datetime(2010,1,1), color='brown', alpha=0.1, lw=0)
  #  ax2.axvspan(datetime(2011,1,1), datetime(2016,1,1), color='brown', alpha=0.1, lw=0)
  ax2.set_xlim(xlim)
  #ax2.set_xticks(xtick)
  #ax2.set_xticklabels(('05','07','09','11','13','15','17','19'))
  ax2.set_ylim(ylim)
  ax2.set_ylabel('Autocorrelation')
  ax2.set_xlabel('Year')
  yend = -0.28
  ft = 14
  hshift = relativedelta(months=3)
  #     ax2.text(mortdate-hshift, 0.05, 'mortality',rotation='vertical',fontsize=ft)
  #     ax2.text(diebackdate-hshift, 0.00, 'dieback',rotation='vertical',fontsize=ft)
  #ax2.text(ewsdate-hshift, -0.12, 'EWS',rotation='vertical',fontsize=ft)

  #     ax2.annotate('', xy=(mortdate, ylim[0]), 
  #                 xytext=(mortdate, yend),arrowprops=arrowprops)
  #     ax2.annotate('', xy=(diebackdate, ylim[0]), 
  #                 xytext=(diebackdate, yend),arrowprops=arrowprops)
  #     ax2.annotate('', xy=(ewsdate, ylim[0]), 
  #                 xytext=(ewsdate, yend),arrowprops=arrowprops)
  ax2.legend(loc=9,ncol=2)
  #     ax2.text(xpos, 0.63, '(b)',fontsize=20)

# This is my attempt at determining appropriate values for the discount factors - it takes a really long time. 

We're ranking based on likelihood as per recommendations in the literature, but the forward filtering calculates residuals at each step (one could also use MAE or RMSE or something similar - when running forwardFilteringM you'd just need an extra line to write the error *e* (=Y-f) to an array and return it

In [80]:
def fit_delta(mod,gap,s=0.98):
    start=s
    end=0.9999

    strend = start
    etrend = end
    
    sregn = start
    eregn = end
    
    sseas = start
    eseas = end
    
    


    for b in [0.005,0.001]:
        trend=np.arange(strend,min(etrend,0.9999),b)
        regn=np.arange(sregn,min(end,eregn),b)
        seas=np.arange(sseas,min(eseas,end),b)
        

        poss_deltas=np.array(np.meshgrid(trend, regn, seas)).T.reshape(-1,3)

        poss_deltas=poss_deltas[np.all(poss_deltas<=1,axis=1)]

        
        
        like=[]
        
        for smurf in np.arange(0,poss_deltas.shape[0]): 
          deltas = poss_deltas[smurf]
          
          M=Model(mod.Y,mod.X,mod.rseas,deltas)
          f = forwardFilteringM(M,gap)
         
          like.append(f["slik"])



        loglike=[]
        for arr in like:
            loglike.append(np.nansum(np.log(arr[int(730/gap):])))
        
        

        score=np.zeros(poss_deltas.shape[0])

        for i in range(poss_deltas.shape[0]):
            
            score[np.argsort(loglike)[i]] +=poss_deltas.shape[0]-i

        rank=poss_deltas[np.argsort(score)]
        

        trend=np.arange(strend,etrend,b)
        regn=np.arange(sregn,eregn,b)
        seas=np.arange(sseas,eseas,b)
        


        strend = rank[0][0]-b
        etrend = rank[0][0]+b
        
        sregn = rank[0][1]-b
        eregn = rank[0][1]+b
        
        sseas = rank[0][2]-b
        eseas = rank[0][2]+b
        
       
        


        print(rank[:5],np.sort(score)[-5:])
    
    return rank[0] 

In [82]:
## this is what I wase using to run things quickly ##
# NDVI is a series of filtered NDVI values
# weather is our (normalised) weather anomalies
# gap is obs frequency of the sat data
# delta is the 3 delta values to use - the ones i came up with are in my thesis
# if fit then we're running the delta finding function and using those values (with lower bound for the deltas as s)
# 0.98 is good for the daily stuff, 
# i'd leave a wider range for less frequent data, maybe adjust "b" in that function to search a bit more coarsely




def run_paper(NDVI,weather,gap=1,delta=[1,1,1],fit=False,s=0.98): 
    
    N = NDVI.values
    fill_value = -999

    CLM = weather.values
    AvgCLM = ave_weather.values


    date0 = weather.index[0]

    anCLM = weather.values

    # center NDVI time series
    N[N==fill_value] = np.nan
    Y = N[1:]-np.nanmean(N) 

    # use two seasonal harmonic components
    rseas = [1,2] 

    # include lag-1 centerred NDVI and precipitation in the regression module 
    X = np.column_stack((N[:-1]-np.nanmean(N),anCLM[:-1,])) 

    # set up model and run forward filtering
    delta = delta
    M = Model(Y,X,rseas,delta)
    
    if fit:
        delta = fit_delta(M,gap,s) 
    
    M = Model(Y,X,rseas,delta)
    FF = forwardFilteringM(M,gap)

    # model likelihood
    slik = FF.get('slik')

    # extract estimates on the coefficient corresponding to lag-1 NDVI
    vid = 2 # index of autocorrelation
    sm = FF.get('sm')[vid,:] # mean of autocorrelation
    sC = FF.get('sC')[vid,vid,:] # variance of autocorrelation
    snu = FF.get('snu') # degree of freedom

    # plot Fig. 1 in the manuscript
    #ax=PlotEWS(N,date0,sm,sC,snu,gap)
    return FF,delta,date0

In [3]:
###DONT RUN THE NEXT CELL IF YOU WANT TO KEEP THIS ###

In [87]:
dates

,Map_no,Name,Age,Lat,Long,SHAPE_Area,ArableRestorationSites
0,23.0,Stonehenge_2005,2005.0,51.178922,-1.838294,49.99,Stonehenge_2005
1,22.0,Stonehenge_2007_2016,2007.0,51.176233,-1.844025,38.40,Stonehenge_2007_2016
2,25.0,Stonehenge_2010,2010.0,51.186129,-1.848284,30.64,Stonehenge_2010
3,21.0,Stonehenge_2009,2009.0,51.174443,-1.851183,25.59,Stonehenge_2009
4,26.0,Stonehenge_2011,2011.0,51.191528,-1.851128,25.55,Stonehenge_2011
5,24.0,Stonehenge_2002_2017,2002.0,51.187837,-1.789613,17.27,Stonehenge_2002_2017
6,18.0,Stonehenge_2000_2017a,2000.0,51.180671,-1.805383,16.39,Stonehenge_2000_2017
7,NaN,NaN,2007.0,NaN,NaN,NaN,NaN


In [ ]:
# NDVI - dataframe - need index = date of collection as datetime, NDVI in column called NDVI (or change NDVI.NDVI in filtered) 
# weather - (daily) weather columns should be "temp" and "percip", index is datetime again 
# ave_weather - long term averages - index is day of year
#Change the start_date depending on year of establishment
# NDVI is always input as a pandas Series with dates as the index

# if you're using the 16 day composite there's a bit more to do with the weather - that's commented out
start_date=2017

NDVI=NDVI[NDVI.index.year>=start_date]
weather=weather[weather.index.year>=start_date]

gap=1 

#calculating weather anomalies

weather[["t_anom","p_anom"]]=weather[["temp","precip"]]-ave_weather.loc[weather.index.dayofyear].values

weather[["t_anom","p_anom"]]=weather[["t_anom","p_anom"]]/np.std(weather[["t_anom","p_anom"]])


filtered=ff_iterdenoise(NDVI.NDVI, gap=1, frequency=2, pad=0.1,iterations=15)  #running the filtering function



s,g=get_con(filtered.filtered,90) #identifying the run of non-nan values to start at

f_filt=filtered.filtered[filtered.index>=s].copy()

f_filt=f_filt[f_filt.index<=weather.index[-1]] #in case you have more 

W=weather[weather.index>=s].copy()

prob=datetime(2020,1,1)

f_filt=f_filt[f_filt.index<prob]

W=W[W.index<prob]

W=W.drop(["temp","precip"],axis=1)


FF, delter,date0 =run_paper(f_filt,W,gap=1,delta=[1,1,1],fit=True,s=0.98)

# extract estimates on the coefficient corresponding to lag-1 NDVI
vid = 2 # index of autocorrelation
sm = FF.get('sm')[vid,:] # mean of autocorrelation
sC = FF.get('sC')[vid,vid,:] # variance of autocorrelation
snu = FF.get('snu') # degree of freedom

PlotEWS(f_filt,date0,sm,sC,snu,gap)

#### for the 16 day composite I did this #####

# sixteen_d.NDVI is our 16 day ndvi (pandas) time  series in the same form as NDVI above


# gap=16
#   filt_sd=ff_iterdenoise(sixteen_d.NDVI, gap=16, frequency=2, pad=0.15 ,iterations=12)

#   sixteen_d.loc[sixteen_d.NDVI.isna(),'NDVI']=-999

#   sd=weather.join(sixteen_d)

#   ind=sd[sd.NDVI.isna()==False].index

#   for i in np.arange(0,len(ind)-1):
#       sd.loc[ind[i],"heat"]=sd[ind[i]:ind[i+1]].t_anom.mean()
#       sd.loc[ind[i],"rain"]=sd[ind[i]:ind[i+1]].p_anom.mean()
      
#   sd.loc[ind[-1],"rain"]=sd[ind[-1]:sd.index[-1]].p_anom.mean()
#   sd.loc[ind[-1],"heat"]=sd[ind[-1]:sd.index[-1]].t_anom.mean()

#   sd_weather=sd[["heat","rain"]].copy().dropna()

# FF, delter,date0 =run_paper(filt_sd.filtered, #I think?
#                             sd_weather,gap=gap,delta=[1,1,1],fit=True,s=0.98)

# vid = 2 # index of autocorrelation
# sm = FF.get('sm')[vid,:] # mean of autocorrelation
# sC = FF.get('sC')[vid,vid,:] # variance of autocorrelation
# snu = FF.get('snu') # degree of freedom
